# Build a SageMaker Pipeline to train and deploy a BERT-Based text classifier
* Define and run a pipeline using a directed acyclic graph (DAG) with specific pipeline parameters and model hyper-parameters
* Define a processing step that cleans, balances, transforms, and splits our dataset into train, validation, and test dataset
* Define a training step that trains a model using the train and validation datasets
* Define a processing step that evaluates the trained model's performance on the test dataset
* Define a register model step that creates a model package from the trained model
* Define a conditional step that checks the model's performance and conditionally registers the model for deployment

**Terminology**

This notebook focuses on the following features of Amazon SageMaker Pipelines:

* **Pipelines** - a directed acyclic graph (DAG) of steps and conditions to orchestrate SageMaker jobs and resource creation
* **Processing job steps** - a simplified, managed experience on SageMaker to run data processing workloads, such as feature engineering, data validation, model evaluation, and model explainability
* **Training job steps** - an iterative process that teaches a model to make predictions on new data by presenting examples from a training dataset
* **Conditional step execution** - provides conditional execution of branches in a pipeline
* **Registering models** - register a model in a model registry to create a deployable models in Amazon SageMaker
* **Parameterized pipeline executions** - allows pipeline executions to vary by supplied parameters
* **Model endpoint** - hosts the model as a REST endpoint to serve predictions from new data

**BERT Pipeline**

The pipeline that you will create follows a typical machine learning application pattern of pre-processing, training, evaluation, and model registration.

In the processing step, you will perform feature engineering to transform the `review_body` text into BERT embeddings using the pre-trained BERT model and split the dataset into train, validation and test files. The transformed dataset is stored in a feature store. To optimize for Tensorflow training, the transformed dataset files are saved using the TFRecord format in Amazon S3.

In the training step, you will fine-tune the BERT model to the customer reviews dataset and add a new classification layer to predict the `sentiment` for a given `review_body`.

In the evaluation step, you will take the trained model and a test dataset as input, and produce a JSON file containing classification evaluation metrics.

In the condition step, you will register the trained model if the accuracy of the model, as determined by our evaluation step, exceeds a given threshold value. 

![](bert_sagemaker_pipeline.png)


In [ ]:
# please ignore warning messages during the installation
!pip install --disable-pip-version-check -q sagemaker==2.35.0

import os, sagemaker, logging, boto3, sagemaker, pandas as pd, json, botocore
from botocore.exceptions import ClientError

config = botocore.config.Config(user_agent_extra='dlai-pds/c2/w3')

# low-level service client of the boto3 session
sm = boto3.client(service_name='sagemaker', config=config)
sm_runtime = boto3.client('sagemaker-runtime', config=config)
sess = sagemaker.Session(sagemaker_client=sm, sagemaker_runtime_client=sm_runtime)

bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = sess.boto_region_name

#Setup pipeline Name
import time
timestamp = int(time.time())
pipeline_name = 'BERT-pipeline-{}'.format(timestamp)

## Configure the dataset and processing step

In [ ]:
#CONFIGURE PROCESSING STEP
#Refer aws pipeline: https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-parameters.html

raw_input_data_s3_uri = 's3://dlai-practical-data-science/data/raw/'

#For the pipeline workflow you will need to create workflow parameters of a specific type: integer, string, or float.
from sagemaker.workflow.parameters import (ParameterInteger,ParameterString,ParameterFloat,)

#set the parameters for the processing step.
processing_instance_type = ParameterString(name="ProcessingInstanceType", default_value="ml.c5.2xlarge")
processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
train_split_percentage = ParameterFloat(name="TrainSplitPercentage", default_value=0.90,)
validation_split_percentage = ParameterFloat(name="ValidationSplitPercentage", default_value=0.05,)
test_split_percentage = ParameterFloat(name="TestSplitPercentage", default_value=0.05,)
balance_dataset = ParameterString(name="BalanceDataset", default_value="True",)
max_seq_length = ParameterInteger(name="MaxSeqLength", default_value=128,)
feature_store_offline_prefix = ParameterString(name="FeatureStoreOfflinePrefix", default_value="reviews-feature-store-" + str(timestamp),)
feature_group_name = ParameterString(name="FeatureGroupName", default_value="reviews-feature-group-" + str(timestamp))
input_data = ParameterString(name="InputData",default_value=raw_input_data_s3_uri,)

#Setting up scikit-learn-based processor to construct a ProcessingStep
from sagemaker.sklearn.processing import SKLearnProcessor
processor = SKLearnProcessor(framework_version='0.23-1',role=role,instance_type=processing_instance_type,
    instance_count=processing_instance_count, env={'AWS_DEFAULT_REGION': region},)

#configure input channel for and output channel from processing step
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

processing_inputs=[
    ProcessingInput(input_name='raw-input-data', source=input_data,destination='/opt/ml/processing/input/data/', s3_data_distribution_type='ShardedByS3Key')]

processing_outputs=[
    ProcessingOutput(output_name='sentiment-train', source='/opt/ml/processing/output/sentiment/train', s3_upload_mode='EndOfJob'),
    ProcessingOutput(output_name='sentiment-validation', source='/opt/ml/processing/output/sentiment/validation', s3_upload_mode='EndOfJob'),
    ProcessingOutput(output_name='sentiment-test', source='/opt/ml/processing/output/sentiment/test', s3_upload_mode='EndOfJob')]        

# similar to a processor instance's run method, use the processor instance to construct a ProcessingStep, along with the input and output channels and the code that will be executed when the pipeline invokes pipeline execution
processing_step = ProcessingStep(
    name='Processing', code='./c2w3/src/prepare_data.py', processor=processor,inputs=processing_inputs,
    outputs=processing_outputs,
    job_arguments=['--train-split-percentage', str(train_split_percentage.default_value),                   
                   '--validation-split-percentage', str(validation_split_percentage.default_value),
                   '--test-split-percentage', str(test_split_percentage.default_value),
                   '--balance-dataset', str(balance_dataset.default_value),
                   '--max-seq-length', str(max_seq_length.default_value),                   
                   '--feature-store-offline-prefix', str(feature_store_offline_prefix.default_value),
                   '--feature-group-name', str(feature_group_name.default_value)])        

print(processing_step)

In [ ]:
# print out the list of the processing job properties
print(json.dumps(processing_step.properties.__dict__, indent=4, sort_keys=True, default=str))

#Pull the channel sentiment-train from the output configuration of the processing job:
print(json.dumps(processing_step.properties.ProcessingOutputConfig.Outputs['sentiment-train'].__dict__, 
                 indent=4, sort_keys=True, default=str))

#print out attributes of the S3 output path related to the sentiment-train output channel:
print(json.dumps(processing_step.properties.ProcessingOutputConfig.Outputs['sentiment-train'].S3Output.S3Uri.__dict__,
                 indent=4, sort_keys=True, default=str))

#These objects can be passed into the next steps of the workflow.

#you can pull the arguments of the processing step with the corresponding function. The result is in the dictionary format
processing_step.arguments.keys()

#Pull and review processing inputs from the arguments of the processing step:
processing_step.arguments['ProcessingInputs']

## Configure training step

In [41]:
#Setup the parameters for the workflow
freeze_bert_layer = ParameterString(name="FreezeBertLayer",default_value="False",)
epochs = ParameterInteger(name="Epochs", default_value=3)    
learning_rate = ParameterFloat(name="LearningRate", default_value=0.00001) 
train_batch_size = ParameterInteger(name="TrainBatchSize", default_value=64)
train_steps_per_epoch = ParameterInteger(name="TrainStepsPerEpoch", default_value=50)
validation_batch_size = ParameterInteger(name="ValidationBatchSize", default_value=64)
validation_steps_per_epoch = ParameterInteger(name="ValidationStepsPerEpoch", default_value=50)
seed = ParameterInteger(name="Seed", default_value=42)
run_validation = ParameterString(name="RunValidation", default_value="True",)
train_instance_count = ParameterInteger(name="TrainInstanceCount", default_value=1)
train_instance_type = ParameterString(name="TrainInstanceType", default_value="ml.c5.9xlarge")
train_volume_size = ParameterInteger(name="TrainVolumeSize", default_value=256) 
input_mode = ParameterString(name="InputMode", default_value="File",)

#Setup the dictionary that will be passed into the hyperparameters argument
hyperparameters={'max_seq_length': max_seq_length, 'freeze_bert_layer': freeze_bert_layer,'epochs': epochs,
    'learning_rate': learning_rate, 'train_batch_size': train_batch_size, 
    'train_steps_per_epoch': train_steps_per_epoch,'validation_batch_size': validation_batch_size, 
    'validation_steps_per_epoch': validation_steps_per_epoch,'seed': seed, 'run_validation': run_validation }

#Configure model-evaluation metrics; Choose loss and accuracy as the evaluation metrics. For example, these sample log lines...
metric_definitions = [{'Name': 'validation:loss', 'Regex': 'val_loss: ([0-9.]+)'}, {'Name': 'validation:accuracy', 'Regex': 'val_acc: ([0-9.]+)'},]

#CONFIGURE PYTORCH ESTIMATOR
#A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to model_dir so that it can be hosted later.
from sagemaker.pytorch import PyTorch as PyTorchEstimator

estimator = PyTorchEstimator(
    entry_point='train.py',source_dir='c2w3/src', role=role,instance_count=train_instance_count, 
    instance_type=train_instance_type, volume_size=train_volume_size,
    py_version='py3', framework_version='1.6.0', hyperparameters=hyperparameters,
    metric_definitions=metric_definitions, input_mode=input_mode)

#Setup pipeline step caching
#Step signature caching allows SageMaker Pipelines, before executing a step, to find a previous execution of a step that was called using the same arguments. Cache hit gets created if the previous execution is found. Then during execution instead of recomputing the step, pipelines propagates the values from the cache hit. details on SageMaker Pipeline step caching: https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-caching.html
from sagemaker.workflow.steps import CacheConfig
cache_config = CacheConfig(enable_caching=True, expire_after="PT1H") # PT1H represents `one hour`

#configure the TrainingStep calling the outputs of the processing step:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep

training_step = TrainingStep(
    name='Train',estimator=estimator,
    inputs={
        'train': TrainingInput(
            s3_data=processing_step.properties.ProcessingOutputConfig.Outputs['sentiment-train'].S3Output.S3Uri,content_type='text/csv'),
        'validation': TrainingInput(
            s3_data=processing_step.properties.ProcessingOutputConfig.Outputs['sentiment-validation'].S3Output.S3Uri, content_type='text/csv')}, 
    cache_config=cache_config)
print(training_step)

TrainingStep(name='Train', step_type=<StepTypeEnum.TRAINING: 'Training'>)


In [ ]:
#print out attributes of the training step properties; The attributes match the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) response object.
processing_step.properties.__dict__

<a name='c2w3-3.'></a>
## 3. Configure model-evaluation step

First, develop an evaluation script that will be specified in the model evaluation processing step. The evaluation script users the trained model and the test dataset to produce a JSON file with classification evaluation metrics such as accuracy.

After pipeline execution, you will examine the resulting `evaluation.json` for analysis.

The evaluation script performs the following steps:
* loads in the model
* reads in the test data
* issues a bunch of predictions against the test data
* builds a classification report, including accuracy
* saves the evaluation report to the evaluation directory

In [43]:
#Create an instance of the `SKLearnProcessor` to run our evaluation script as a scikit-learn-based SageMaker processing job.
from sagemaker.sklearn.processing import SKLearnProcessor

evaluation_processor = SKLearnProcessor(
    framework_version='0.23-1', role=role, instance_type=processing_instance_type,
    instance_count=processing_instance_count, env={'AWS_DEFAULT_REGION': region}, max_runtime_in_seconds=7200)

#Setup the output PropertyFile.
from sagemaker.workflow.properties import PropertyFile
evaluation_report = PropertyFile(name='EvaluationReport', output_name='metrics', path='evaluation.json')

#Use the processor instance to construct a ProcessingStep, along with the input and output channels and the code that will be executed when the pipeline invokes pipeline execution. This is very similar to a processor instance's run method.
from sagemaker.processing import ProcessingInput, ProcessingOutput

evaluation_step = ProcessingStep(
    name='EvaluateModel', processor=evaluation_processor, code='c2w3/src/evaluate_model_metrics.py',
    inputs=[
        ProcessingInput(source=training_step.properties.ModelArtifacts.S3ModelArtifacts, destination='/opt/ml/processing/input/model'),
        ProcessingInput(source=processing_step.properties.ProcessingOutputConfig.Outputs['sentiment-test'].S3Output.S3Uri, destination='/opt/ml/processing/input/data')],
    outputs=[ProcessingOutput(output_name='metrics', s3_upload_mode='EndOfJob', source='/opt/ml/processing/output/metrics/'),],
    job_arguments=['--max-seq-length', str(max_seq_length.default_value),], 
    property_files=[evaluation_report],)

<a name='c2w3-4.'></a>
# 4. Configure and register model step
### 4.1. Configure the model for deployment

Use the estimator instance that was used for the training step to construct an instance of `RegisterModel`. The result of executing `RegisterModel` in a pipeline is a model package. A model package is a reusable model artifacts abstraction that packages all ingredients necessary for inference. Primarily, it consists of an inference specification that defines the inference image to use along with an optional model weights location.

A model package group is a collection of model packages. You can create a model package group for a specific ML business problem, and you can keep adding versions/model packages into it. Typically, customers are expected to create a ModelPackageGroup for a SageMaker workflow pipeline so that they can keep adding versions/model packages to the group for every workflow pipeline run.

The construction of `RegisterModel` is very similar to an estimator instance's `register` method, for those familiar with the existing Python SDK.

In particular, you will pass in the `S3ModelArtifacts` from the `training_step` properties.

Of note, here you will be provided a specific model package group name which will be used in the Model Registry and Continuous Integration/Continuous Deployment (CI/CD) work later on. Let's setup the variables.

In [44]:
model_approval_status = ParameterString(name="ModelApprovalStatus", default_value="PendingManualApproval")
deploy_instance_type = ParameterString(name="DeployInstanceType", default_value="ml.m5.large")
deploy_instance_count = ParameterInteger(name="DeployInstanceCount", default_value=1)

model_package_group_name = f"BERT-Reviews-{timestamp}"
print(model_package_group_name)

#Configure the ModelMetrics to be stored as metadata.
from sagemaker.model_metrics import MetricsSource, ModelMetrics 

model_metrics = ModelMetrics(model_statistics=MetricsSource(s3_uri="{}/evaluation.json".format(
                evaluation_step.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]),content_type="application/json"))
print(model_metrics)

#Define deployment image for inference.
inference_image_uri = sagemaker.image_uris.retrieve(framework="pytorch", region=region,version="1.6.0", 
                                                    py_version="py36", instance_type=deploy_instance_type, image_scope="inference")
print(inference_image_uri)

#Register the model for deployment
from sagemaker.workflow.step_collections import RegisterModel

register_step = RegisterModel(name="RegisterModel", estimator=estimator, image_uri=inference_image_uri, 
    model_data=training_step.properties.ModelArtifacts.S3ModelArtifacts, content_types=["application/jsonlines"],
    response_types=["application/jsonlines"], inference_instances=[deploy_instance_type],
    transform_instances=[deploy_instance_type], # batch transform is not used in this lab
    model_package_group_name=model_package_group_name, approval_status=model_approval_status, model_metrics=model_metrics)

BERT-Reviews-1658899925
763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.6.0-cpu-py36


<a name='c2w3-5.'></a>
## 5. Create model for deployment step

In [45]:
#Configure model for deployment.
from sagemaker.model import Model

model_name = 'bert-model-{}'.format(timestamp)

model = Model(name=model_name, image_uri=inference_image_uri,
    model_data=training_step.properties.ModelArtifacts.S3ModelArtifacts, sagemaker_session=sess, role=role,)

#Now configure create model input:
from sagemaker.inputs import CreateModelInput
create_inputs = CreateModelInput(instance_type=deploy_instance_type, )

#Configure create model step for the workflow.
from sagemaker.inputs import CreateModelInput
create_inputs = CreateModelInput(instance_type=deploy_instance_type, )

#Configure create model step for the workflow
from sagemaker.workflow.steps import CreateModelStep
create_step = CreateModelStep(name="CreateModel", model=model, inputs=create_inputs, )

<a name='c2w3-6.'></a>
# 6. Check accuracy condition step

Finally, you would like to only register this model if the accuracy of the model, as determined by our evaluation step `evaluation_step`, exceeded some value. A `ConditionStep` allows for pipelines to support conditional execution in the pipeline DAG based on conditions of step properties. 
Below, you will:
* define a minimum accuracy value as a parameter
* define a `ConditionGreaterThan` on the accuracy value found in the output of the evaluation step, `evaluation_step`.
* use the condition in the list of conditions in a `ConditionStep`
* pass the `RegisterModel` step collection into the `if_steps` of the `ConditionStep`

In [46]:
min_accuracy_value = ParameterFloat(name="MinAccuracyValue", default_value=0.33) # random choice from three classes

from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import (ConditionStep, JsonGet,)

minimum_accuracy_condition = ConditionGreaterThanOrEqualTo(
    left=JsonGet(step=evaluation_step, property_file=evaluation_report, json_path="metrics.accuracy.value",),
    right=min_accuracy_value) # minimum accuracy threshold

minimum_accuracy_condition_step = ConditionStep(
    name="AccuracyCondition",conditions=[minimum_accuracy_condition],
    if_steps=[register_step, create_step], # successfully exceeded or equaled the minimum accuracy, continue with model registration
    else_steps=[],) # did not exceed the minimum accuracy, the model will not be registered

<a name='c2w3-7.'></a>
# 7. Create pipeline
<a name='c2w3-7.1.'></a>
### 7.1. Define a pipeline of parameters, steps, and conditions
Let's tie it all up into a workflow pipeline so you can execute it, and even schedule it. A pipeline requires a `name`, `parameters`, and `steps`. Names must be unique within an `(account, region)` pair so you can append the timestamp to the name to reduce the chance of name conflict.

Note:
* All the parameters used in the definitions must be present.
* Steps passed into the pipeline need not be in the order of execution. The SageMaker workflow service will resolve the _data dependency_ DAG as steps the execution complete.
* Steps must be unique to either pipeline step list or a single condition step if/else list.

In [ ]:
from sagemaker.workflow.pipeline import Pipeline

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        input_data, processing_instance_count, processing_instance_type, max_seq_length, balance_dataset,
        train_split_percentage, validation_split_percentage, test_split_percentage, feature_store_offline_prefix,
        feature_group_name, epochs, learning_rate, train_batch_size, train_steps_per_epoch, validation_batch_size,
        validation_steps_per_epoch, freeze_bert_layer, seed, train_instance_count, train_instance_type, train_volume_size,        
        input_mode, run_validation, min_accuracy_value, model_approval_status, deploy_instance_type, deploy_instance_count],
    steps=[processing_step, training_step, evaluation_step, minimum_accuracy_condition_step],sagemaker_session=sess,)

#Let's examine the JSON of the pipeline definition that meets the SageMaker Workflow Pipeline DSL specification.By examining the definition, you are also confirming that the pipeline was well-defined, and that the parameters and step properties resolve correctly.
import json
from pprint import pprint
definition = json.loads(pipeline.definition())
pprint(definition)

In [49]:
#Create pipeline
response = pipeline.create(role_arn=role)

pipeline_arn = response["PipelineArn"]
print(pipeline_arn)

#Start Pipeline: submit our pipeline definition to the Amazon SageMaker Pipeline service. 
execution = pipeline.start(
    parameters=dict(
        InputData=raw_input_data_s3_uri, ProcessingInstanceCount=1, ProcessingInstanceType='ml.c5.2xlarge',
        MaxSeqLength=128, BalanceDataset='True', TrainSplitPercentage=0.9, ValidationSplitPercentage=0.05,
        TestSplitPercentage=0.05, FeatureStoreOfflinePrefix='reviews-feature-store-'+str(timestamp),
        FeatureGroupName='reviews-feature-group-'+str(timestamp), Epochs=3, LearningRate=0.000012, TrainBatchSize=64,
        TrainStepsPerEpoch=50, ValidationBatchSize=64, ValidationStepsPerEpoch=64, FreezeBertLayer='False', Seed=42,         
        TrainInstanceCount=1, TrainInstanceType='ml.c5.9xlarge', TrainVolumeSize=256, InputMode='File', RunValidation='True',
        MinAccuracyValue=0.01, ModelApprovalStatus='PendingManualApproval',  DeployInstanceType='ml.m5.large', DeployInstanceCount=1 ))
print(execution.arn)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


arn:aws:sagemaker:us-east-1:170235698766:pipeline/bert-pipeline-1658899925
arn:aws:sagemaker:us-east-1:170235698766:pipeline/bert-pipeline-1658899925/execution/6y432kk0ojbt


In [ ]:
#describe execution instance and list the steps in the execution to find out more about the execution.
from pprint import pprint
execution_run = execution.describe()
pipeline_execution_arn = execution_run['PipelineExecutionArn']
execution_run_name = execution_run['PipelineExecutionDisplayName']
pprint(execution_run_name)
print(pipeline_execution_arn)
print(execution_run)

#Describe completed pipeline
import time
time.sleep(30)
execution.list_steps()

In [ ]:
#observing the pipeline execution summary and waiting for the execution status to change from Executing to Succeeded.
#%%time
import time
from pprint import pprint

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)['PipelineExecutionSummaries']
pipeline_execution_status = executions_response[0]['PipelineExecutionStatus']
print(pipeline_execution_status)

while pipeline_execution_status=='Executing':
    try:
        executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)['PipelineExecutionSummaries']
        pipeline_execution_status = executions_response[0]['PipelineExecutionStatus']
    except Exception as e:
        print('Please wait...')
        time.sleep(30)    
    
pprint(executions_response)

pipeline_execution_status = executions_response[0]['PipelineExecutionStatus']
print(pipeline_execution_status)
pipeline_execution_arn = executions_response[0]['PipelineExecutionArn']
print(pipeline_execution_arn)

<a name='c2w3-8.'></a>
# 8. Evaluate the model

In [ ]:
#Describe evaluation metrics: Examine the resulting model evaluation after the pipeline completes. Download the resulting evaluation.json file from S3 and print the report.
# pull and print the pipeline artifacts
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer

viz = LineageTableVisualizer(sagemaker.session.Session())

processing_job_name=None
training_job_name=None

for execution_step in reversed(execution.list_steps()):
    pprint(execution_step)
    if execution_step['StepName'] == 'Processing':
        processing_job_name=execution_step['Metadata']['ProcessingJob']['Arn'].split('/')[-1]
        print('Processing job name: {}'.format(processing_job_name))
        display(viz.show(processing_job_name=processing_job_name))
    elif execution_step['StepName'] == 'Train':
        training_job_name=execution_step['Metadata']['TrainingJob']['Arn'].split('/')[-1]
        print('Training job name: {}'.format(training_job_name))
        display(viz.show(training_job_name=training_job_name))
    else:
        display(viz.show(pipeline_execution_step=execution_step))
        time.sleep(5)

# list the files in the resulting output S3 path
!aws s3 ls --recursive $transform_output_s3_uri

#Download the evaluation report and print the accuracy.
from pprint import pprint
evaluation_json = sagemaker.s3.S3Downloader.read_file("{}/evaluation.json".format(evaluation_metrics_s3_uri))
pprint(json.loads(evaluation_json))

<a name='c2w3-9.'></a>
# 9. Deploy and test the model

In [ ]:
#Approve trained model: The pipeline created a model package version within the specified model package group and an approval status of `PendingManualApproval`.  This requires a separate step to manually approve the model before deploying to production. You can approve the model using the SageMaker Studio UI or programmatically as shown below.

#Get the model package ARN
for execution_step in execution.list_steps():
    if execution_step['StepName'] == 'RegisterModel':
        model_package_arn = execution_step['Metadata']['RegisterModel']['Arn']
        break
print(model_package_arn)

#Update the model package with the Approved status to prepare for deployment
model_package_update_response = sm.update_model_package(ModelPackageArn=model_package_arn, ModelApprovalStatus="Approved",)
pprint(model_package_update_response)

#Get the model ARN and the model name from it.
for execution_step in execution.list_steps():
    print(execution_step['StepName'])
    if execution_step['StepName'] == 'CreateModel':
        model_arn = execution_step['Metadata']['Model']['Arn']
        break
print(model_arn)

model_name = model_arn.split('/')[-1]
print(model_name)

#Configure the endpoint.
endpoint_config_name = 'bert-model-epc-{}'.format(timestamp)
print(endpoint_config_name)

create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{'InstanceType':'ml.m5.xlarge', 'InitialVariantWeight':1, 'InitialInstanceCount':1, 'ModelName': model_name, 
                         'VariantName':'AllTraffic'}])

#Create the endpoint.
pipeline_endpoint_name = 'bert-model-ep-{}'.format(timestamp)
print("EndpointName={}".format(pipeline_endpoint_name))

create_endpoint_response = sm.create_endpoint(EndpointName=pipeline_endpoint_name, EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

#review endpoint link
from IPython.core.display import display, HTML
display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST Endpoint</a></b>'.format(region, pipeline_endpoint_name)))

In [ ]:
#wait till deployed
%%time
while True:
    try: 
        waiter = sm.get_waiter('endpoint_in_service')
        print('Waiting for endpoint to be in `InService`...')
        waiter.wait(EndpointName=pipeline_endpoint_name)
        break;
    except:
        print('Waiting for endpoint...')
        endpoint_status = sm.describe_endpoint(EndpointName=pipeline_endpoint_name)['EndpointStatus']
        print('Endpoint status: {}'.format(endpoint_status))
        if endpoint_status == 'Failed':
            break
        time.sleep(30)  
print('Endpoint deployed.')

In [ ]:
#Test model: Predict the sentiment with review_body samples and review the result:
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONLinesSerializer
from sagemaker.deserializers import JSONLinesDeserializer

inputs = [
    {"features": ["I love this product!"]},
    {"features": ["OK, but not great."]},
    {"features": ["This is not the right product."]},]

predictor = Predictor(endpoint_name=pipeline_endpoint_name, serializer=JSONLinesSerializer(), deserializer=JSONLinesDeserializer(),sagemaker_session=sess)
predicted_classes = predictor.predict(inputs)

for predicted_class in predicted_classes:
    print("Predicted class {} with probability {}".format(predicted_class['predicted_label'], predicted_class['probability']))
    
#SageMaker Studio provides a rich set of features to visually inspect SageMaker resources including pipelines, training jobs, and endpoints. 